In [1]:
!pip install medmnist

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com


In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [2]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v3.0.1 @ https://github.com/MedMNIST/MedMNIST/


In [3]:
data_flag = 'breastmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [4]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz


In [5]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 546
    Root location: /Users/yuqi/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1': 'normal, benign'}
    Number of samples: {'train': 546, 'val': 78, 'test': 156}
    Description: The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
    License: CC BY 4.0
Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 156
    Root location: /Users/yuqi/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning

## Data Preprocessing

# We first compare their singular values

## We first use standard SVD to check their singular values

In [6]:
from SVD import standard_svd
svd_standard={}
singular_values={}
x_train={}
y_train={}
for i in range(10):
    x_train[str(i)],y_train[str(i)]=train_dataset[i]
    svd_standard[str(i)],singular_values[str(i)]=standard_svd(x_train[str(i)][0])

## then we try our first sketching matrix

In [11]:
!pip install utils

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com


## Uniform Sketching Matrices

In [7]:
uniform_matrices={}
from sketching import uniform_sketching_matrix
from QB_decomposition_fixed_precision import randQB_FP_auto
for i in range (10):
    uniform_matrices[str(i)]=uniform_sketching_matrix(28, 12)
#get their QB decomposition
#rank_approximated={}
Q_uniform={}
B_uniform={}
iteration_uniform={}
error_uniform={}
for i in range(10):
    Q,B,error,iteration=randQB_FP_auto(x_train[str(i)][0],0.1,4,4,uniform_matrices[str(i)])
    Q_uniform[str(i)]=Q
    B_uniform[str(i)]=B
    error_uniform[str(i)]=error
    iteration_uniform[str(i)]=iteration

fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge


In [8]:
print(error_uniform)

{'0': 87.92549878097101, '1': 246.41146612394908, '2': 86.12221312491273, '3': 107.93778157287234, '4': 60.88505713642677, '5': 136.18199060280406, '6': 163.24084083228504, '7': 357.9721299720461, '8': 395.21218466138527, '9': 132.04162185790744}


In [12]:
#then we do SVM based on the new matrix
rand_singular_values={}
for i in range(10):
    _, rand_singular_values[str(i)]=standard_svd(B)

In [ ]:
#compare singular values
import numpy as np
from scipy.linalg import svd
from numpy.linalg import qr

def randomized_svd(A, k, random_matrix,p=5, n_iter=2):
    """
    Perform Randomized SVD on matrix A.
    
    Parameters:
        A: ndarray, shape (m, n)
           Input matrix to decompose.
        k: int
           Target rank (number of singular values/vectors to compute).
        p: int, optional (default=5)
           Oversampling parameter to improve approximation accuracy.
        n_iter: int, optional (default=2)
           Number of iterations for power iteration (improves accuracy).
           
    Returns:
        U: ndarray, shape (m, k)
           Approximated left singular vectors.
        S: ndarray, shape (k,)
           Approximated singular values.
        V: ndarray, shape (k, n)
           Approximated right singular vectors.
    """
    # Step 1: Random sampling
    m, n = A.shape
   
    Y = A @ random_matrix

    # Step 2: Power iteration (optional, improves accuracy for ill-conditioned matrices)
    for _ in range(n_iter):
        Y = A @ (A.T @ Y)

    # Step 3: QR decomposition
    Q, _ = qr(Y, mode='reduced')

    # Step 4: Project matrix to lower-dimensional space
    B = Q.T @ A

    # Step 5: Compute SVD on the smaller matrix
    U_tilde, S, Vt = svd(B, full_matrices=False)

    # Step 6: Reconstruct the left singular vectors
    U = Q @ U_tilde

    return U[:, :k], S[:k], Vt[:k, :]


## Gaussian Sketching matrices

In [ ]:
gaussian_matrices={}
from sketching import gaussian_sketching_matrix
from QB_decomposition_fixed_precision import randQB_FP_auto
for i in range (10):
    gaussian_matrices[str(i)]=gaussian_sketching_matrix(28, 12)
#get their QB decomposition
#rank_approximated={}
    

fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge
fail to converge


In [ ]:
# Here we use blockwise computation
Q_gaussian={}
B_gaussian={}
error_gaussion={}
iteration_gaussian={}
#print(gaussian_matrices[str(0)])
for i in range(10):
    Q,B,error,iteration=randQB_FP_auto(train_dataset[i][0][0],0.01,4,4,gaussian_matrices[str(i)])
    #rank_approximated[str(i)]=k
    Q_gaussian[str(i)]=Q
    B_gaussian[str(i)]=B
    error_gaussion[(str(i))]=error
    iteration_gaussian[str(i)]=iteration


## Radamecher Sketching Matrices

## Johnson-Linderstrauss Transform Code

In [ ]:
from jlt.linearMapping import calculate_R
from QB_decomposition_fixed_precision import randQB_FP_auto
JLT_matrices={}
for i in range(10):
    JLT_matrices[str(i)]=calculate_R(28,k=None,s=1,random_seed=21,swr=True)
    

In [ ]:
#

## Clarkson_Woodruff

In [ ]:
from sketching import cwt_sketch_matrix


# Visualization

In [ ]:
#We try to do some boxplots
#Considering relevant Error